In [ ]:
# File: trampoline.ipynb
# Code: Claude Code and Codex
# Review: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
from frontend import App

# create an app
app = App.create("trampoline")

# create a square sheet mesh
V, F = app.mesh.square(res=128, ex=[1, 0, 0], ey=[0, 0, 1])
app.asset.add.tri("sheet", V, F)

# create an armadillo tetrahedral mesh
V, F, T = app.mesh.preset("armadillo").decimate(30000).tetrahedralize().normalize()
app.asset.add.tet("armadillo", V, F, T)

# create a scene
scene = app.scene.create()

# add sheet as trampoline with dynamic area coloring
sheet = scene.add("sheet").dyn_color("area", 1.0).direction([1, 0, 0], [0, 0, 1])

# pin the four edges of the sheet
sheet.pin(sheet.grab([-1, 0, 0]) + sheet.grab([1, 0, 0]))
sheet.param.set("young-mod", 1e4).set("strain-limit", 0.01)

# add armadillo falling onto trampoline with velocity
armadillo = scene.add("armadillo").color(0.75, 0.75, 0.75).scale(0.75).rotate(180, "y")
armadillo.at(0, 1, 0).jitter().velocity(0, -5, 0)
armadillo.param.set("density", 6e3).set("friction", 0.5)

# add invisible walls to contain the simulation
gap = 0.025
scene.add.invisible.wall([1 + gap, 0, 0], [-1, 0, 0])
scene.add.invisible.wall([-1 - gap, 0, 0], [1, 0, 0])
scene.add.invisible.wall([0, 0, 1 + gap], [0, 0, -1])
scene.add.invisible.wall([0, 0, -1 - gap], [0, 0, 1])

# compile the scene and report stats
scene = scene.build().report()

# preview the initial scene
scene.preview()

In [ ]:
# create a new session with the compiled scene
session = app.session.create(scene)

# set session parameters
if app.ci:
    session.param.set("frames", 60)
else:
    session.param.set("frames", 120)

# build this session
session = session.build()

In [ ]:
# start the simulation and live-preview the results
session.start().preview()

# also show simulation logs in realtime
session.stream()

In [ ]:
# create an animation from the simulation results
session.animate()

In [ ]:
# export the animation to file
session.export.animation()

In [ ]:
# this is for CI
if app.ci:
    assert session.finished()